In [1]:
##@ All pip installs

!pip install -Uqq pip llama-index llama-index-vector-stores-chroma llama-index-llms-huggingface-api llama-index-embeddings-huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 46.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 83.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 65.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 808.7/808.7 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.1/253.1 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101

In [12]:
!git config --global credential.helper store

In [13]:
##@ HF login 

from huggingface_hub import login 

login()

In [14]:
#@ Here we have a basic agent workflow creating a basic agentic calculator
##@ I know it's a overkill of an calculator 😵 but well whatever we just learning here

from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from llama_index.core.agent.workflow import AgentWorkflow, ToolCallResult, AgentStream

def add(a: int, b:int)-> int:
    """Add two ints """
    return a + b


def mul(a: int, b: int)-> int:
    """Multiply two ints"""
    return a*b


def sub(a:int, b:int) -> int:
    """Subtract two ints"""
    return a-b

def div(a:int, b:int)-> int:
    """Divide two ints"""
    return a/b


llm = HuggingFaceInferenceAPI(model_name= "Qwen/Qwen2.5-Coder-32B-Instruct")


agent = AgentWorkflow.from_tools_or_functions(
    tools_or_functions=[add, mul, sub, div],
    llm=llm,
    system_prompt="You're a math agent that can add, multiply, divide and subtract using provided tools"
)

In [6]:
handler = agent.run("What is (4+4)/2*66?")
async for ev in handler.stream_events():
    if isinstance(ev, ToolCallResult):
        print("")
        print("Called Tool: ", ev.tool_name, ev.tool_kwargs, "=>", ev.tool_output)
    elif isinstance(ev, AgentStream):
        print(ev.delta, end="", flush=True)

resp = await handler
resp

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: add
Action Input: {"a": 4, "b": 4}
Called Tool:  add {'a': 4, 'b': 4} => 8
Thought: I now have the result of 4 + 4, which is 8. Next, I need to divide this result by 2.
Action: div
Action Input: {'a': 8, 'b': 2}
Called Tool:  div {'a': 8, 'b': 2} => 4.0
Thought: I now have the result of 8 / 2, which is 4.0. Next, I need to multiply this result by 66.
Action: mul
Action Input: {'a': 4, 'b': 66}
Called Tool:  mul {'a': 4, 'b': 66} => 264
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: 264

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='264')]), tool_calls=[ToolCallResult(tool_name='add', tool_kwargs={'a': 4, 'b': 4}, tool_id='23c083fb-da96-4b7d-9cdf-f833911d5833', tool_output=ToolOutput(content='8', tool_name='add', raw_input={'args': (), 'kwargs': {'a': 4, 'b': 4}}, raw_output=8, is_error=False), return_direct=False), ToolCallResult(tool_name='div', tool_kwargs={'a': 8, 'b': 2}, tool_id='e091c9c0-6610-4553-8b54-788f7e5367f2', tool_output=ToolOutput(content='4.0', tool_name='div', raw_input={'args': (), 'kwargs': {'a': 8, 'b': 2}}, raw_output=4.0, is_error=False), return_direct=False), ToolCallResult(tool_name='mul', tool_kwargs={'a': 4, 'b': 66}, tool_id='6f5a50fe-7d7f-45be-857a-ae83ce66e30b', tool_output=ToolOutput(content='264', tool_name='mul', raw_input={'args': (), 'kwargs': {'a': 4, 'b': 66}}, raw_output=264, is_error=False), return_direct=False)], raw=ChatCompletionStrea

In [15]:
#@ Likewise we can also pass state and context to the agent


from llama_index.core.workflow import Context

cntx = Context(agent)

response = await agent.run("My name is Firoj", ctx = cntx)
response

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='Nice to meet you, Firoj! How can I assist you today?')]), tool_calls=[ToolCallResult(tool_name='None', tool_kwargs={}, tool_id='d058ff2e-a837-4b94-bb6a-7bb9e8858983', tool_output=ToolOutput(content='Tool None not found. Please select a tool that is available.', tool_name='None', raw_input={}, raw_output=None, is_error=True), return_direct=False)], raw=ChatCompletionStreamOutput(choices=[ChatCompletionStreamOutputChoice(delta=ChatCompletionStreamOutputDelta(role='assistant', content='?', tool_call_id=None, tool_calls=None), index=0, finish_reason=None, logprobs=None)], created=1745065382, id='', model='Qwen/Qwen2.5-Coder-32B-Instruct', system_fingerprint='3.2.1-sha-4d28897', usage=None, object='chat.completion.chunk'), current_agent_name='Agent')

In [9]:
response = await agent.run("I want to build an agent", ctx= cntx)
response

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='That sounds exciting! Could you please provide more details about the type of agent you want to build? Are you referring to a software agent, a chatbot, a robot, or something else? Additionally, what are the main functionalities or goals you have in mind for this agent?')]), tool_calls=[], raw=ChatCompletionStreamOutput(choices=[ChatCompletionStreamOutputChoice(delta=ChatCompletionStreamOutputDelta(role='assistant', content='?', tool_call_id=None, tool_calls=None), index=0, finish_reason=None, logprobs=None)], created=1745065447, id='', model='Qwen/Qwen2.5-Coder-32B-Instruct', system_fingerprint='3.2.1-sha-4d28897', usage=None, object='chat.completion.chunk'), current_agent_name='Agent')

In [16]:
response = await agent.run("What was my name and what am I planning to do?", ctx = cntx)
response

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text="Your name is Firoj. You haven't mentioned what you are planning to do, so I don't have that information. How can I assist you today?")]), tool_calls=[ToolCallResult(tool_name='None', tool_kwargs={}, tool_id='fc5a545b-f652-4e98-9e05-0f943ee651b5', tool_output=ToolOutput(content='Tool None not found. Please select a tool that is available.', tool_name='None', raw_input={}, raw_output=None, is_error=True), return_direct=False)], raw=ChatCompletionStreamOutput(choices=[ChatCompletionStreamOutputChoice(delta=ChatCompletionStreamOutputDelta(role='assistant', content='?', tool_call_id=None, tool_calls=None), index=0, finish_reason=None, logprobs=None)], created=1745065574, id='', model='Qwen/Qwen2.5-Coder-32B-Instruct', system_fingerprint='3.2.1-sha-4d28897', usage=None, object='chat.completion.chunk'), current_agent_name='Agent')

### Creating RAG Agents with QueryEngineTools

In [18]:
import chromadb


from llama_index.core import VectorStoreIndex
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.tools import QueryEngineTool
from llama_index.vector_stores.chroma import ChromaVectorStore


##@ Creating a vector store
db = chromadb.PersistentClient(path="./alfred_chroma_db")
chroma_collection = db.get_or_create_collection("alfred")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

##@ Query Engine 
embed_model = HuggingFaceEmbedding(model_name = "BAAI/bge-small-en-v1.5")
llm = HuggingFaceInferenceAPI(model_name = "Qwen/Qwen2.5-Coder-32B-Instruct")
index = VectorStoreIndex.from_vector_store(
    vector_store = vector_store, embed_model = embed_model
)

query_engine = index.as_query_engine(llm=llm)
query_engine_tool = QueryEngineTool.from_defaults(
    query_engine = query_engine, 
    name = "personas",
    description = "descriptions for various types of personas",
    return_direct = False
)

##! Here we have our RAG agent 
query_engine_agent = AgentWorkflow.from_tools_or_functions(
    tools_or_functions = [query_engine_tool], 
    llm= llm, 
    system_prompt= "You are a helpful assistant that has access to a database containing persona descriptions."
)

In [19]:
handler = query_engine_agent.run(
    "Search the database for 'science fiction' and return some persona descriptions. "
)

async for ev in handler.stream_events():
    if isinstance(ev, ToolCallResult):
        print("")
        print("Called Tool: ", ev.tool_name, ev.tool_kwargs, "=>", ev.tool_output)
    elif isinstance(ev, AgentStream):
        print(ev.delta, end="", flush=True)

resp = await handler
resp

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: personas
Action Input: {"input": "science fiction"}
Called Tool:  personas {'input': 'science fiction'} => Empty Response
Thought: I received an empty response from the personas tool, which means there are no specific persona descriptions for 'science fiction' in the current database. I'll need to provide a general answer based on common personas associated with the genre.
Answer: In the context of science fiction, some common persona descriptions include:

1. **Futuristic Scientist**: A highly intelligent and innovative individual who specializes in advanced scientific research, often pushing the boundaries of what is known and developing groundbreaking technologies.

2. **Space Explorer**: An adventurous and brave individual who ventures into the unknown depths of space, exploring new planets and encountering alien life forms.

3. **Cyberpunk Hacker**: A skilled and res

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='In the context of science fiction, some common persona descriptions include:\n\n1. **Futuristic Scientist**: A highly intelligent and innovative individual who specializes in advanced scientific research, often pushing the boundaries of what is known and developing groundbreaking technologies.\n\n2. **Space Explorer**: An adventurous and brave individual who ventures into the unknown depths of space, exploring new planets and encountering alien life forms.\n\n3. **Cyberpunk Hacker**: A skilled and resourceful individual who navigates through complex digital systems, often using their expertise to uncover secrets or protect against cyber threats.\n\n4. **Robotic Engineer**: A dedicated and meticulous individual who designs and builds robots, often focusing on creating machines that can assist humans or perform tasks in hazardous environments.\n\n5.

### Creating multi-agent systems

In [20]:
from llama_index.core.agent.workflow import (
    AgentWorkflow,
    ReActAgent,
)


# Define some tools
def add(a: int, b: int) -> int:
    """Add two numbers."""
    return a + b


def subtract(a: int, b: int) -> int:
    """Subtract two numbers."""
    return a - b


# Create agent configs
# NOTE: we can use FunctionAgent or ReActAgent here.
# FunctionAgent works for LLMs with a function calling API.
# ReActAgent works for any LLM.
calculator_agent = ReActAgent(
    name="calculator",
    description="Performs basic arithmetic operations",
    system_prompt="You are a calculator assistant. Use your tools for any math operation.",
    tools=[add, subtract],
    llm=llm,
)

query_agent = ReActAgent(
    name="info_lookup",
    description="Looks up information about XYZ",
    system_prompt="Use your tool to query a RAG system to answer information about XYZ",
    tools=[query_engine_tool],
    llm=llm,
)

# Create and run the workflow
agent = AgentWorkflow(agents=[calculator_agent, query_agent], root_agent="calculator")

# Run the system
handler = agent.run(user_msg="Can you add 5 and 3?, also look out for the researcher")

In [21]:
async for ev in handler.stream_events():
    if isinstance(ev, ToolCallResult):
        print("")
        print("Called tool: ", ev.tool_name, ev.tool_kwargs, "=>", ev.tool_output)
    elif isinstance(ev, AgentStream):  # showing the thought process
        print(ev.delta, end="", flush=True)

resp = await handler
resp

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: add
Action Input: {"a": 5, "b": 3}
Called tool:  add {'a': 5, 'b': 3} => 8
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The sum of 5 and 3 is 8.

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='The sum of 5 and 3 is 8.')]), tool_calls=[ToolCallResult(tool_name='add', tool_kwargs={'a': 5, 'b': 3}, tool_id='9ccec60b-3ad7-4510-82cd-fe6ef80ece0e', tool_output=ToolOutput(content='8', tool_name='add', raw_input={'args': (), 'kwargs': {'a': 5, 'b': 3}}, raw_output=8, is_error=False), return_direct=False)], raw=ChatCompletionStreamOutput(choices=[ChatCompletionStreamOutputChoice(delta=ChatCompletionStreamOutputDelta(role='assistant', content='.', tool_call_id=None, tool_calls=None), index=0, finish_reason=None, logprobs=None)], created=1745066623, id='', model='Qwen/Qwen2.5-Coder-32B-Instruct', system_fingerprint='3.2.1-sha-4d28897', usage=None, object='chat.completion.chunk'), current_agent_name='calculator')